In [2]:
%load_ext Cython

import arviz as az
import corner
import cython
import emcee
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import platform
import pymc3 as pm
import theano
import theano.tensor as tt

print('Running on PyMC3 v{}'.format(pm.__version__))
az.style.use('arviz-darkgrid')

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython
Running on PyMC3 v3.9.0


If we have a model function or probability distribution that relies on external code and may even be wrapped C code, what can we do? If we want to pass `PyMC3` variables or distributions to these external functions, we need to be clever, since these external functions probably expect floats.

Also, to perform gradient-based step sampling, the models or likelihoods we pass will need a specified gradient. If the model uses Theano operators, then autodiff is automatic; otherwise, if our model is a black box, we will not know what the gradients are.

Defining a model or likelihood that uses our "black box" external function is possible: we need to create a custom Theano Op.

This example creates a simple model and log-likelihood using `Cython`. This is merely an example where we call external `C` code, but we could easily do this if we were calling external Python code.

We will write our external function in `C` and use the GNU Scientific Library `GSL` to perform operations, showing that we can use external `C` libraries.

The below seems to work to use Cython. We need to tell it which files to link, though.

In [22]:
%%cython -I/usr/local/include -L/usr/local/lib -lgsl -lgslcblas -lm
# -I : list of include directories
# -L : list of library directories
# -l : add library to link the extension against
# %%cython -I/usr/include -L/usr/lib/x86_64-linux-gnu -lgsl -lgslcblas -lm
import cython 
cimport cython

import numpy as np 
cimport numpy as np